In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv("./input/train.csv")
x = train_df.drop('label', axis=1)
y = pd.get_dummies(train_df['label'])
x = np.array(x)
y = np.array(y)

# 訓練とテストデータに分割
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [3]:
import chainer
from chainer import functions as F
from chainer import links as L

class Model(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(Model, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_units)
            self.l2 = L.Linear(n_units, n_units)
            self.l3 = L.Linear(n_units, n_out)
          
    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        z = F.relu(self.l3(h2))
        return z

#モデルのインスタンスの作成
n_units = 200
n_classes = 10
model = L.Classifier(Model(n_units, n_classes))

In [4]:
import numpy as np
from chainer.dataset import concat_examples
from chainer.cuda import to_cpu
from chainer import iterators
from chainer import optimizers
from chainer.datasets import mnist
train,test = mnist.get_mnist(withlabel=True, ndim=1)
batch_size = 128

train_iter = iterators.SerialIterator(train, batch_size)
test_iter = iterators.SerialIterator(test, batch_size,repeat=False,shuffle=False)
optimizer = optimizers.Adam()
optimizer.setup(model)
epoch_num = 10

i=0
while train_iter.epoch < epoch_num:
    i +=1
    batch = train_iter.next()
    x, y = concat_examples(batch)
  #作成したモデルにデータを入力して、出力y_predを得るコード
    y_pred = model.predictor(x)
  #出力y_predと正解yからロスを計算するコード
    loss = F.softmax_cross_entropy(y_pred, y)
  #ロスのVariableのbackward()メソッドを呼ぶコード
    model.cleargrads()
    loss.backward()
  #Optimizerのupdate()メソッドを呼ぶコード
    optimizer.update()
    
    if train_iter.is_new_epoch:
        
        test_losses = []
        test_accuracies = []
        
        while True:
            test_batch = test_iter.next()
            x_test, y_test = concat_examples(test_batch)
            
            y_test_pred = model.predictor(x_test)
            
            test_loss = F.softmax_cross_entropy(y_test_pred, y_test)
            test_losses.append(test_loss.data)
        
            test_accuracy = F.accuracy(y_test_pred, y_test)
            test_accuracies.append(test_accuracy.data)
            
            if test_iter.is_new_epoch:
                test_iter.epoch = 0
                test_iter.current_position = 0
                test_iter.is_new_epoch = False
                test_iter._pushed_position = None
                break
                
        print('val_loss:{:.04f} val_acc:{:.04f}'.format(np.mean(test_losses),\
                                                        np.mean(test_accuracies)))

val_loss:0.5883 val_acc:0.7744
val_loss:0.5540 val_acc:0.7804
val_loss:0.5420 val_acc:0.7828
val_loss:0.5372 val_acc:0.7847
val_loss:0.5360 val_acc:0.7840
val_loss:0.5267 val_acc:0.7841
val_loss:0.5307 val_acc:0.7864
val_loss:0.5270 val_acc:0.7874
val_loss:0.5314 val_acc:0.7849
val_loss:0.5389 val_acc:0.7847
